In [348]:
import numpy as np
import sympy as sp
from sympy.physics.quantum import TensorProduct

# Define constants for Fibonacci anyons
phi = (1 + sp.sqrt(5)) / 2
phi2 = phi**(-1)

# Define the F matrix
F = sp.Matrix([[phi2, sp.sqrt(phi2)], [sp.sqrt(phi2), -phi2]]).evalf()

# Define the R matrix
R = np.array([sp.exp(-4 * sp.pi * 1j / 5), sp.exp(3 * sp.pi * 1j / 5)])

# Define the basis states
basis_states = [(1, 1), (1, 'tau'), ('tau', 1), ('tau', 'tau')]

# Define the index mapping
basis_index = {(1, 1): 0, (1, 'tau'): 1, ('tau', 1): 2, ('tau', 'tau'): 3}

# Define the Fibonacci anyon fusion rules
def fibonacci_fusion_rule(anyon1, anyon2):
    if anyon1 == 'tau' and anyon2 == 'tau':
        return [1, 'tau']
    if (anyon1 == 'tau' and anyon2 == 1) or (anyon1 == 1 and anyon2 == 'tau'):
        return ['tau']
    if anyon1 == 1 and anyon2 == 1:
        return [1]
    else:
        return 0

#F-matrix
def Fma(a, b, c, d, x, y):    
    if x in fibonacci_fusion_rule(a,b) and y in fibonacci_fusion_rule(b,c):
        if d in fibonacci_fusion_rule(x,c) and d in fibonacci_fusion_rule(y,a):
            # Check the consistency of fusion rules
            if a == 1 or b == 1 or c == 1 or d == 1:
                return 1
            if x == 1 and y == 1:
                return phi2
            if (x == 1 and y == 'tau') or (x == 'tau' and y == 1):
                return sp.sqrt(phi2)
            if x == 'tau' and y == 'tau':
                return -phi2
        else:
            return 0
    else:
        return 0

In [349]:
###Sigma 1
sigma1 = sp.Matrix(np.zeros((5, 5), dtype=complex)).evalf()
for i in range(5):
    if i in {0, 1}:
        sigma1[i, i] = R[0]
    elif i in {2, 3, 4}:
        sigma1[i, i] = R[1]
        
sigma1f = sigma1.evalf()
# Print the sigma1 matrix
#print(sigma1)
#sp.pprint(sigma1)

In [350]:

###Sigma 2
# Initialize the sigma2 matrix with zeros (4x4 matrix)
sigma2 = sp.Matrix(np.zeros((5, 5), dtype=complex))
#F = {(1,1):phi2, (1,'tau'):np.sqrt(phi2), ('tau', 1): np.sqrt(phi2), ('tau', 'tau'): -phi2}

# Function to add terms to the sigma2 matrix
def add_terms(i, x, y, term1, term2, term3, sigma):
    if (x, y) in basis_index:
        sigma[basis_index[(1, y)], i] += term1
        sigma[basis_index[('tau', y)], i] += term2
        sigma[-1,-1] = term3
# Define the computation for each basis state
for i, (x, y) in enumerate(basis_states):
    for j, (z, Rz) in enumerate(zip([1, 'tau'], R)):
        term1 = Fma('tau','tau','tau','tau',x, z) * Rz * Fma('tau','tau','tau','tau',z, 1)
        term2 = Fma('tau','tau','tau','tau',x, z) * Rz * Fma('tau','tau','tau','tau',z, 'tau')
        term3 = R[1] 
        add_terms(i, x, y, term1, term2,term3, sigma2)    
sigma2f = sp.simplify(sigma2).evalf()
#print(sigma2)
#sp.pprint(sigma2)  

In [351]:
###Sigma 3
# Initialize the sigma2 matrix with zeros
sigma3 = sp.Matrix(np.zeros((5, 5), dtype=complex))

# Function to add terms to the sigma2 matrix
def add_terms2(i, x, y, term3,term4,sigma):
    if (x, y) in basis_index:
        sigma[basis_index[(x, y)],i] += term3
        sigma[-1,i] += term4

# Define the computation for each basis state
for i, (x, y) in enumerate(basis_states):
        for j, (z, Rz) in enumerate(zip([1, 'tau'], R)):
                term3 = Fma('tau','tau',y,x, z, 'tau') * Rz * Fma('tau','tau',y,x,z,'tau')
                term4 = Fma('tau','tau',y,x,z, 'tau') * Rz * Fma('tau','tau',y,x,z,1)
                add_terms2(i, x, y, term3,term4, sigma3)
                
# Function to add terms to the sigma2 matrix
def add_terms5(x, y, term5,term6,sigma):
    if (x, y) in basis_index:
        sigma[-1,-1] += term5
        sigma[3,4] += term6
          
for z, Rz in zip([1,'tau'],R):
        term5 = Fma('tau','tau','tau','tau',z,1) * Rz * Fma('tau','tau','tau','tau',z,1)
        term6 = Fma('tau','tau','tau','tau',z,1) * Rz * Fma('tau','tau','tau','tau',z,'tau') 
        add_terms5(x, y, term5,term6, sigma3)
              
sigma3f = sp.simplify(sigma3).evalf()                        
#sp.pprint(sigma3[:,4])  

In [352]:
###Sigma 4
# Initialize the sigma2 matrix with zeros (4x4 matrix)
sigma4 = sp.Matrix(np.zeros((4, 4), dtype=complex))
#F = {(1,1):phi2, (1,'tau'):np.sqrt(phi2), ('tau', 1): np.sqrt(phi2), ('tau', 'tau'): -phi2}

# Function to add terms to the sigma2 matrix
def add_terms1(i, x, y, term3, term4, sigma):
    if (x, y) in basis_index:
        sigma[basis_index[(x, 1)], i] += term3
        sigma[basis_index[(x, 'tau')], i] += term4

# Define the computation for each basis state
for i, (x, y) in enumerate(basis_states):
    for j, (z, Rz) in enumerate(zip([1, 'tau'], R)):
        term3 = F[y != 1, j] * Rz * F[j, 0]
        term4 = F[y != 1, j] * Rz * F[j, 1]
        add_terms1(i, x, y, term3, term4, sigma4)    
                
rows, cols = sigma4.shape
extended_sigma4 = sp.Matrix(np.zeros((rows + 1, cols + 1), dtype=complex))
extended_sigma4[:rows, :cols] = sigma4
extended_sigma4[4,4] = R[1]
sigma4f = extended_sigma4.evalf()
#sp.pprint(sigma4f[:,0])  

In [353]:
###Sigma 5
sigma5 = sp.Matrix(np.zeros((5, 5), dtype=complex))
for i in range(5):
    if i in {0, 2}:
        sigma5[i, i] = R[0]
    elif i in {1, 3, 4}:
        sigma5[i, i] = R[1]

sigma5f = sigma5.evalf()
#sp.pprint(sigma5[:,4])

In [354]:

# Compute sigma1 * sigma2 * sigma1
product1 = sp.simplify(sigma1f@sigma2f@sigma1f)

# Compute sigma2 * sigma1 * sigma2
product2 = sp.simplify(sigma2f@sigma1f@sigma2f)

# Compute sigma2 * sigma3 * sigma2
product11 = sp.simplify(sigma2f@sigma3f@sigma2f)

# Compute sigma3 * sigma2 * sigma3
product12 = sp.simplify(sigma3f@sigma2f@sigma3f)

# Compute sigma2 * sigma3 * sigma2
product13 = sp.simplify(sigma3f@sigma4f@sigma3f)

# Compute sigma3 * sigma2 * sigma3
product14 = sp.simplify(sigma4f@sigma3f@sigma4f)

# Compute sigma4 * sigma5 * sigma4
product3 = sp.simplify(sigma5f@sigma4f@sigma5f)

# Compute sigma5 * sigma4 * sigma5
product4 = sp.simplify(sigma4f@sigma5f@sigma4f)

# Compute sigma1 * sigma4
product5 = sp.simplify(sigma1f@sigma4f)

# Compute sigma4 * sigma1
product6 = sp.simplify(sigma4f@sigma1f)

# Compute sigma1 * sigma5
product7 = sp.simplify(sigma1f@sigma5f)

# Compute sigma5 * sigma1
product8 = sp.simplify(sigma5f@sigma1f)

# Compute sigma2 * sigma5
product9 = sp.simplify(sigma2f@sigma5f)

# Compute sigma5 * sigma2
product10 = sp.simplify(sigma5f@sigma2f)

In [355]:
def compare_matrices(matrix1, matrix2,tolerance=1e-9):
    if matrix1.shape != matrix2.shape:
        return False, {"reason": "Matrices have different shapes"}

    unequal_elements = {}
    are_equal = True
    
    for i in range(matrix1.shape[0]):
        for j in range(matrix1.shape[1]):
            if np.abs(matrix1[i, j] - matrix2[i, j]) >= tolerance:
                are_equal = False
                unequal_elements[(i, j)] = (matrix1[i, j], matrix2[i, j])
    
    if are_equal:
        return True, {}
    else:
        return False, unequal_elements
#sigma_1 = "\u03C3\u2081"
#sigma_2 = "\u03C3\u2082"
#sigma_3 = "\u03C3\u2083"
#sigma_4 = "\u03C3\u2084"
#sigma_5 = "\u03C3\u2085"

# Sigma1 and Sigma2

are_equal, details = compare_matrices(product1, product1)
if are_equal:
    print("\u03C3\u2081\u03C3\u2082\u03C3\u2081 = \u03C3\u2082\u03C3\u2081\u03C3\u2082 is verified")
else:
    print("Matrices are not equal.")
    print("Unequal elements found at:")
    for position, values in details.items():
        print(f"Position {position}: Matrix1 has {values[0]}, Matrix2 has {values[1]}")
        
# Sigma2 and Sigma3

are_equal, details = compare_matrices(product11, product12)
if are_equal:
    print("\u03C3\u2082\u03C3\u2083\u03C3\u2082 = \u03C3\u2083\u03C3\u2082\u03C3\u2083 is verified")
else:
    print("Matrices are not equal.")
    print("Unequal elements found at:")
    for position, values in details.items():
        print(f"Position {position}: Matrix1 has {values[0]}, Matrix2 has {values[1]}")
        
#Sigma 3 and Sigma4

are_equal, details = compare_matrices(product13, product14)
if are_equal:
    print("\u03C3\u2083\u03C3\u2084\u03C3\u2083 = \u03C3\u2084\u03C3\u2083\u03C3\u2084 is verified")
else:
    print("Matrices are not equal.")
    print("Unequal elements found at:")
    for position, values in details.items():
        print(f"Position {position}: Matrix1 has {values[0]}, Matrix2 has {values[1]}")        
        
# Sigma5 and Sigma4

are_equal, details = compare_matrices(product3, product4)
if are_equal:
    print("\u03C3\u2084\u03C3\u2085\u03C3\u2084 = \u03C3\u2085\u03C3\u2084\u03C3\u2085 is verified")
else:
    print("Matrices are not equal.")
    print("Unequal elements found at:")
    for position, values in details.items():
        print(f"Position {position}: Matrix1 has {values[0]}, Matrix2 has {values[1]}")
    
# Sigma1 and Sigma4

are_equal, details = compare_matrices(product5, product6)
if are_equal:
    print("\u03C3\u2081\u03C3\u2084 = \u03C3\u2084\u03C3\u2081 is verified")
else:
    print("Matrices are not equal.")
    print("Unequal elements found at:")
    for position, values in details.items():
        print(f"Position {position}: Matrix1 has {values[0]}, Matrix2 has {values[1]}")
        
# Sigma1 and Sigma5

are_equal, details = compare_matrices(product7, product8)
if are_equal:
    print("\u03C3\u2081\u03C3\u2085 = \u03C3\u2085\u03C3\u2081 is verified")
else:
    print("Matrices are not equal.")
    print("Unequal elements found at:")
    for position, values in details.items():
        print(f"Position {position}: Matrix1 has {values[0]}, Matrix2 has {values[1]}")

# Sigma2 and Sigma5

are_equal, details = compare_matrices(product9, product10)
if are_equal:
    print("\u03C3\u2082\u03C3\u2085 = \u03C3\u2085\u03C3\u2082 is verified")
else:
    print("Matrices are not equal.")
    print("Unequal elements found at:")
    for position, values in details.items():
        print(f"Position {position}: Matrix1 has {values[0]}, Matrix2 has {values[1]}")
    
        

σ₁σ₂σ₁ = σ₂σ₁σ₂ is verified
σ₂σ₃σ₂ = σ₃σ₂σ₃ is verified
σ₃σ₄σ₃ = σ₄σ₃σ₄ is verified
σ₄σ₅σ₄ = σ₅σ₄σ₅ is verified
σ₁σ₄ = σ₄σ₁ is verified
σ₁σ₅ = σ₅σ₁ is verified
σ₂σ₅ = σ₅σ₂ is verified


In [356]:
#Leakage-Free
def Leakagefree(sigma,tol=1e-10):
    sigmaf = sp.simplify(sigma).evalf()
    norm = sp.simplify(((sp.re(sigmaf[-1,-1])**(2)) + (sp.im(sigmaf[-1,-1])**(2)))**(1/2))
    #print(norm)
    if abs(norm - 1 ) < tol:
        return True
    else:
        return False

#Test
test1 = (sigma3f@sigma2f@sigma1f)@(sigma1f@sigma2f@sigma3f)
test2 = (sigma1f@sigma2f@sigma1f)@(sigma5f@sigma4f@sigma5f)@(sigma3f@sigma2f@sigma1f)@(sigma4f@sigma3f@sigma2f)@(sigma5f@sigma4f@sigma3f)
#sp.pprint(sp.simplify(sp.Abs(test2[4,4])))
print(Leakagefree(test1))
print(Leakagefree(test2))

True
True


In [357]:
def set_small_values_to_zero(matrix, tolerance=1e-17):
    mutable_matrix = matrix.as_mutable()  # Convert to mutable matrix
    for i in range(mutable_matrix.rows):
        for j in range(mutable_matrix.cols):
            if abs(sp.re(mutable_matrix[i, j])) < tolerance and abs(sp.im(mutable_matrix[i, j])) < tolerance:
                mutable_matrix[i, j] = 0
            elif abs(sp.re(mutable_matrix[i, j])) > tolerance and abs(sp.im(mutable_matrix[i, j])) < tolerance:
                mutable_matrix[i,j] = sp.re(mutable_matrix[i,j])
            elif abs(sp.re(mutable_matrix[i, j])) < tolerance and abs(sp.im(mutable_matrix[i, j])) > tolerance:
                mutable_matrix[i,j] = sp.im(mutable_matrix[i,j])*sp.I
            elif abs(sp.re(mutable_matrix[i, j])) > tolerance and abs(sp.im(mutable_matrix[i, j])) > tolerance:
                continue
    return mutable_matrix

In [358]:
#Entangling
SWAP = sp.Matrix([[1,0,0,0],[0,0,1,0],[0,1,0,0],[0,0,0,1]])

def are_proportional(matrix1, matrix2, tol=1e-10):
    #matrix1 = set_small_values_to_zero(Matrix1)
    #matrix2 = set_small_values_to_zero(Matrix2)
    if matrix1.shape != matrix2.shape:
        return None, False
    # Calculate the ratios of corresponding elements
    ratios = []
    for i in range(matrix1.shape[0]):
        for j in range(matrix1.shape[1]):
            if matrix1[i,j] != 0:
                ratio = sp.simplify(matrix2[i, j] / matrix1[i, j])
                ratios.append(ratio)
            elif matrix1[i, j] == 0  and matrix2[i,j] == 0:
                continue
            elif matrix1[i, j] == 0 and matrix2[i,j] != 0:
                return None, False  
    # Check if all ratios are the same (within a tolerance for floating-point comparisons)
    #sp.pprint(ratios)
    #sp.pprint(ratios[1:])
    first_ratio = ratios[0]
    if len(ratios) == 1:
        return sp.simplify(first_ratio), True
    elif len(ratios) > 1:
        for ratio in ratios[1:]:
            diff = sp.simplify(first_ratio - ratio).evalf()
            if abs(sp.re(diff)) < tol and abs(sp.im(diff)) < tol:
                return sp.simplify(first_ratio) , True
            else:
                return None, False
    else:
        return None, False

In [359]:
def is_zero_matrix(matrix,tol=1e-20):
    return all(abs(element) < tol for element in matrix)


In [360]:
# Define the Entangling function
def Entangling(sigma,tol=1e-20):
    sigmafinal = set_small_values_to_zero(sp.simplify(sigma))
    if Leakagefree(sigmafinal) == False:
        return False
    comp_basis_4x4 = sigmafinal[:4, :4]
    # Divide into 2x2 matrices
    a11B = comp_basis_4x4[:2, :2]
    a12B = comp_basis_4x4[:2, 2:]
    a21B = comp_basis_4x4[2:, :2]
    a22B = comp_basis_4x4[2:, 2:]
    matrices = [a11B,a12B,a21B,a22B]
    non_zero = []
    for matrix in matrices:
        if is_zero_matrix(matrix):
            continue
        else:
            non_zero.append(matrix)
            
    if len(non_zero) == 1:
        return False, sp.pprint((non_zero[0]))                      
    #sp.pprint(len(non_zero))                    
    props = []
    ratios = []
    for i in range(1, len(non_zero)):
        ratio, prop = are_proportional(non_zero[0], non_zero[i])
        props.append(prop)
        ratios.append(ratio)
    #sp.pprint(ratios)
    #sp.pprint(props)
    if all(props):
        B = non_zero[0]
        A = sp.Matrix([[0, 0], [0, 0]])
        indeces = []
        for elem in non_zero:
            indeces.append(matrices.index(elem))
            
        #sp.pprint(indeces)
        if indeces[0] == 0:
            A[0,0] = 1
        elif indeces[0] == 1:
            A[0,1] = 1
        elif indeces[0] == 2:
            A[1,0] = 1
        elif indeces[0] == 3:
            A[1,1] = 1
        #sp.pprint(len(ratios))
        for p, n in enumerate(indeces[1:]):
            if n == 0:
                A[0,0] = ratios[p]
            elif n == 1:
                A[0,1] = ratios[p]
            elif n == 2:
                A[1,0] = ratios[p]
            elif n == 3:
                A[1,1] = ratios[p] 
        #sp.pprint(A)
        #sp.pprint(B)
        #sp.pprint(comp_basis_4x4[:,0])
        #sp.pprint(sp.simplify(TensorProduct(A,B))[:,0])                     
        equal1, detalles1 = compare_matrices(comp_basis_4x4,sp.simplify(TensorProduct(A,B)))
        if equal1:
            return False
        else:
            return True,detalles1
    else:
        new_4x4 = sp.simplify(SWAP@comp_basis_4x4)
        # Divide into 2x2 matrices
        a11B = new_4x4[:2, :2]
        a12B = new_4x4[:2, 2:]
        a21B = new_4x4[2:, :2]
        a22B = new_4x4[2:, 2:]
        matrices = [a11B,a12B,a21B,a22B]
        non_zero = []
        for matrix in matrices:
            if is_zero_matrix(matrix):
                continue
            else:
                non_zero.append(matrix)
        if len(non_zero) == 1:
            return False, sp.pprint((non_zero[0]))                   
    #sp.pprint(len(non_zero))                    
        props = []
        ratios = []
        for i in range(1, len(non_zero)):
            ratio, prop = are_proportional(non_zero[0], non_zero[i])
            props.append(prop)
            ratios.append(ratio)
    #sp.pprint(len(ratios))
        if all(props):
            B = non_zero[0]
            A = sp.Matrix([[0, 0], [0, 0]])
            indeces = []
            for elem in non_zero:
                indeces.append(matrices.index(elem))
            
            #sp.pprint(indeces)
            if indeces[0] == 0:
                A[0,0] = 1
            elif indeces[0] == 1:
                A[0,1] = 1
            elif indeces[0] == 2:
                A[1,0] = 1
            elif indeces[0] == 3:
                A[1,1] = 1
            #sp.pprint(len(ratios))
            for p, n in enumerate(indeces[1:]):
                if n == 0:
                    A[0,0] = ratios[p]
                elif n == 1:
                    A[0,1] = ratios[p]
                elif n == 2:
                    A[1,0] = ratios[p]
                elif n == 3:
                    A[1,1] = ratios[p] 
            #sp.pprint(A)
            #sp.pprint(B)
            #sp.pprint(new_4x4[:,0])
            #sp.pprint(sp.simplify(TensorProduct(A,B))[:,0])      
            equal2, detalles2 = compare_matrices(new_4x4,sp.simplify(TensorProduct(A,B)))
            if equal2:
                return False
            else:
                return True,detalles2
    
                    
result = Entangling(test1)
print(result)

False


In [361]:
#Pruebas
prueba1 = sigma1f@sigma2f@sigma5f
sp.pprint(Leakagefree(prueba1))
sp.pprint(Entangling(prueba1))

True
False


In [362]:
#Prueba
prueba2 = sigma3f@sigma1f@sigma5f@sigma2f@sigma1f
sp.pprint(Leakagefree(prueba2))
sp.pprint(Entangling(prueba2))
#sp.pprint(sp.Abs(sp.simplify(prueba2)[4,4]))

False
False


In [363]:
#Prueba
prueba3 = sigma1f@sigma2f@sigma3f@sigma5f@sigma4f@sigma4f
sp.pprint(Leakagefree(prueba3))
sp.pprint(Entangling(prueba3))

False
False


In [364]:
#Prueba
prueba4 = sigma5f@sigma2f@sigma3f@sigma5f@sigma2f@sigma4f@sigma3f
sp.pprint(Leakagefree(prueba4))
sp.pprint(Entangling(prueba4))

False
False


In [365]:
#Prueba
prueba5 = sigma5f@sigma5f@sigma5f@sigma3f@sigma4f@sigma4f
sp.pprint(Leakagefree(prueba5))
sp.pprint(Entangling(prueba5))

False
False


In [366]:
#Prueba
prueba6 = sigma2f@sigma2f@sigma3f@sigma2f@sigma4f@sigma5f
sp.pprint(Leakagefree(prueba6))
sp.pprint(Entangling(prueba6))

False
False


In [367]:
#Prueba
prueba7 = sigma4f@sigma2f@sigma3f@sigma5f@sigma4f@sigma4f@sigma5f@sigma1f@sigma2f
sp.pprint(Leakagefree(prueba7))
sp.pprint(Entangling(prueba7))

False
False


In [368]:
#Prueba
prueba8 = sigma1f@sigma2f@sigma3f@sigma5f@sigma4f@sigma4f@sigma1f@sigma3f
sp.pprint(Leakagefree(prueba8))
sp.pprint(Entangling(prueba8))

False
False


In [369]:
#Prueba
prueba9 = sigma1f@sigma2f@sigma3f@sigma5f@sigma4f@sigma5f
sp.pprint(Leakagefree(prueba9))
sp.pprint(Entangling(prueba9))

False
False


In [370]:
#Prueba
prueba10 = sigma1f@sigma2f
sp.pprint(Leakagefree(prueba10))
sp.pprint(Entangling(prueba10))

True
False


In [371]:
#Prueba
prueba11 = sigma1f@sigma2f@sigma3f
sp.pprint(Leakagefree(prueba11))
sp.pprint(Entangling(prueba11))

False
False


In [372]:
#Prueba
prueba12 = sigma1f@sigma2f@sigma3f@sigma4f
sp.pprint(Leakagefree(prueba12))
sp.pprint(Entangling(prueba12))

False
False


In [373]:
#Prueba
prueba13 = sigma1f@sigma2f@sigma3f@sigma4f@sigma5f
sp.pprint(Leakagefree(prueba13))
sp.pprint(Entangling(prueba13))

False
False


In [374]:
#Prueba
prueba14 = sigma1f@sigma4f@sigma4f
sp.pprint(Leakagefree(prueba14))
sp.pprint(Entangling(prueba14))

True
False


In [375]:
#Prueba
prueba15 = sigma2f@sigma2f@sigma1f@sigma5f@sigma3f@sigma4f
sp.pprint(Leakagefree(prueba15))
sp.pprint(Entangling(prueba15))

False
False


In [376]:
#Prueba
prueba16 = sigma2f@sigma2f@sigma3f@sigma4f@sigma3f@sigma4f
sp.pprint(Leakagefree(prueba16))
sp.pprint(Entangling(prueba16))

False
False


In [377]:
#Prueba
prueba17 = sigma5f@sigma4f@sigma3f@sigma2f@sigma1f@sigma5f@sigma4f
sp.pprint(Leakagefree(prueba17))
sp.pprint(Entangling(prueba17))

False
False


In [378]:
#Prueba
prueba18 = sigma1f@sigma5f@sigma4f
sp.pprint(Leakagefree(prueba18))
sp.pprint(Entangling(prueba18))

True
False


In [379]:
#Prueba
prueba19 = sigma1f@sigma3f@sigma2f@sigma3f@sigma5f@sigma4f@sigma1f@sigma5f@sigma3f@sigma4f
sp.pprint(Leakagefree(prueba19))
sp.pprint(Entangling(prueba19))

False
False


In [380]:
#Prueba
prueba20 = sigma1f@sigma2f@sigma5f@sigma5f@sigma3f@sigma4f@sigma2f@sigma4f@sigma5f
sp.pprint(Leakagefree(prueba3))
sp.pprint(Entangling(prueba3))

False
False
